#*************Parameters***************************

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable


In [0]:
%sql
SELECT * FROM workspace.silver.silver_bookings

In [0]:
key_cols = "['booking_id']"
key_cols_list = eval(key_cols)
catalog = "workspace"


In [0]:
key_cols

In [0]:
key_cols_list

###CDC column

In [0]:
cdc_col = "modified_date"

###Backdated Refresh

In [0]:
backdated_refresh = ""

###Source Object

In [0]:
source_object = "silver_bookings"

###Source Schema 

In [0]:
source_schema = "silver"

#Target Schema

In [0]:
target_schema = "gold"

In [0]:
target_object = "FactBookings"

In [0]:
surrogate_key = "DimBookingsKey"

In [0]:
# Catalog Name
catalog = "workspace"

# Source Schema
source_schema = "silver"

# Source Object 
source_object = "silver_bookings"

# CDC Column
cdc_column = "modified_Date"

# Backdated Refresh
backdated_refresh = ""

# Source Fact Table
fact_table = f"{catalog}.{source_schema}.{source_object}"

# Target Schema 
target_schema = "gold"

# Target Object 
target_object = "FactBookings"

# Fact Key Cols List 
fact_key_cols = ["DimPassengersKey","DimFlightsKey","DimAirportsKey","booking_date"]


In [0]:
dimensions = [
    {
        "table": f"{catalog}.{target_schema}.DimPassengers",
        "alias": "DimPassengers",
        "join_keys": [("passenger_id", "passenger_id")]  # (fact_col, dim_col)
    },
    {
        "table": f"{catalog}.{target_schema}.DimFlights",
        "alias": "Flights",
        "join_keys": [("flight_id", "flight_id")]  # (fact_col, dim_col)
    },
    {
        "table": f"{catalog}.{target_schema}.DimAirports",
        "alias": "DimAirports",
        "join_keys": [("airport_id", "airport_id")]  # (fact_col, dim_col)
    },
]

'''
Note : You can use the below code to add more dimensions to the fact table.
 "join_keys": [
            ("CountryID", "CountryID"),
            ("RegionID", "RegionID")
        ]
'''


# Columns you want to keep from Fact table (besides the surrogate keys)
fact_columns = ["amount","booking_date","modifiedDate"]

In [0]:
dimensions

In [0]:
# dimensions=eval(dimensions)

In [0]:
dimensions

In [0]:
fact_columns[0]

#Last Load Date

In [0]:
target_object,target_schema,catalog

In [0]:
%sql
select max(modified_date) from workspace.gold.FactBookings

In [0]:
spark.sql(f"SELECT max({cdc_column}) FROM {catalog}.{target_schema}.{target_object}").collect()[0][0]


In [0]:
cdc_column

In [0]:
# No Back Dated Refresh
if len(backdated_refresh) == 0:
  
  # If Table Exists In The Destination
  if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):

    last_load = spark.sql(f"SELECT max({cdc_column}) FROM workspace.{target_schema}.{target_object}").collect()[0][0]
    
  else:
    last_load = "1900-01-01 00:00:00"

# Yes Back Dated Refresh
else:
  last_load = backdated_refresh

# Test The Last Load 
last_load

# if len(backdated_refresh)==0:
#     if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
#         last_load = spark.sql(f"select max({cdc_col}) from {catalog}.{target_schema}.{target_object}").collect()[0][0]
#     else:
#         last_load = "1900-01-01 00:00:00"
# else:
#     last_load = backdated_refresh

        

**DYNAMIC FACT QUERY [BRING KEYS]**

In [0]:
fact_table

In [0]:
dimensions

In [0]:
def generate_fact_query_incremental(fact_table, dimensions, fact_columns, cdc_column, processing_date):
    fact_alias = "f"
    
    # Base columns to select
    select_cols = [f"{fact_alias}.{col}" for col in fact_columns]

    # Build joins dynamically
    join_clauses = []
    for dim in dimensions:
        table_full = dim["table"]
        alias = dim["alias"]
        table_name = table_full.split('.')[-1]
        surrogate_key = f"{alias}.{table_name}Key"
        select_cols.append(surrogate_key)

        # Build ON clause
        on_conditions = [
            f"{fact_alias}.{fk} = {alias}.{dk}" for fk, dk in dim["join_keys"]
        ]
        join_clause = f"LEFT JOIN {table_full} {alias} ON " + " AND ".join(on_conditions)
        join_clauses.append(join_clause)

    # Final SELECT and JOIN clauses
    select_clause = ",\n    ".join(select_cols)
    joins = "\n".join(join_clauses)

    # WHERE clause for incremental filtering
    where_clause = f"{fact_alias}.{cdc_column} >= DATE('{last_load}')"

    # Final query
    query = f"""
SELECT
    {select_clause}
FROM {fact_table} {fact_alias}
{joins}
WHERE {where_clause}
""".strip()

    return query

In [0]:
query = generate_fact_query_incremental(fact_table, dimensions, fact_columns, cdc_column, last_load)

#DF_FACT

In [0]:
df_fact = spark.sql(query)

#UPSERT

In [0]:
# Fact Key Columns Merge Condition
fact_key_cols_str = " AND ".join([f"src.{col} = trg.{col}" for col in fact_key_cols])
fact_key_cols_str

In [0]:
#Testing
df_src = spark.sql(f"select * from {source_schema}.{source_object} where {cdc_col}>='{last_load}'")

In [0]:
df_src.display()

#Old VS NEW

In [0]:
key_cols_list

In [0]:
if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
    key_cols_string_incremental = ", ".join(key_cols_list)

    df_trg = spark.sql(f"""select {key_cols_string_incremental}, {surrogate_key}, create_date, update_date from {catalog}.{target_schema}.{target_object}""")

else:
    #key columns string for Initial
    key_cols_string_init = [f"'' AS {i}" for i in key_cols_list]
    key_cols_string_init = ", ".join(key_cols_string_init)
    df_trg = spark.sql(f"""select {key_cols_string_init}, CAST('0' AS INT) AS {surrogate_key},CAST('1900-01-01 00:00:00' AS timestamp) AS create_date,CAST('1900-01-01 00:00:00' AS timestamp) AS update_date WHERE 1=0""")


In [0]:
df_trg.display()

#Join Condition

In [0]:
join_condition = ' AND '.join([f"src.{i} = trg.{i}" for i in key_cols_list])

In [0]:
join_condition

In [0]:
df_src.display()

In [0]:
df_trg.display()

In [0]:
df_src.createOrReplaceTempView("src")
df_trg.createOrReplaceTempView("trg")

df_join = spark.sql(f"""SELECT src.*, trg.{surrogate_key}, trg.create_date, trg.update_date FROM src LEFT JOIN trg ON {join_condition}""")

In [0]:
df_join.display()

#Old REcords

In [0]:
df_old = df_join.filter(col(f"{surrogate_key}").isNotNull())
df_old.display()
df_new = df_join.filter(col(f"{surrogate_key}").isNull())
df_new.display()
df_old.count()

#Preparing the DF_OLD

In [0]:
df_old_enr = df_old.withColumn("update_date", current_timestamp())

In [0]:
df_old_enr.display()

# Preparing the DF_NEW

In [0]:
df_new.display()

In [0]:
if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
    max_surrogate_key = spark.sql(f"""select max({surrogate_key}) from {catalog}.{target_schema}.{target_object}""").collect()[0][0]

    df_new_enr = df_new.withColumn(f'{surrogate_key}', lit(max_surrogate_key) + lit(1)+ monotonically_increasing_id())\
                       .withColumn('create_date', current_timestamp())\
                       .withColumn('update_date', current_timestamp())
else:
    max_surrogate_key = 0
    df_new_enr = df_new.withColumn(f'{surrogate_key}', lit(max_surrogate_key) + lit(1)+ monotonically_increasing_id())\
                        .withColumn('create_date', current_timestamp())\
                        .withColumn('update_date', current_timestamp())
df_new_enr.display()

In [0]:
df_old_enr.display()

#Union old and New records

In [0]:
df_union = df_old_enr.unionByName(df_new_enr)

In [0]:
df_union.display()

###Modified_date column was the original column to compare target_object "Dim_flights"

In [0]:
if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
    dlt_obj = DeltaTable.forName(spark, f"{catalog}.{target_schema}.{target_object}")
    dlt_obj.alias("trg").merge(df_union.alias("src"),
        f"trg.{surrogate_key} = src.{surrogate_key}")\
        .whenMatchedUpdateAll(condition=f"src.{cdc_col} = trg.{cdc_col}")\
        .whenNotMatchedInsertAll()\
        .execute()
else:
    df_union.write.format("delta")\
        .mode("append")\
        .saveAsTable(f"{catalog}.{target_schema}.{target_object}")

#Validation in the Gold Layer

In [0]:
%sql
select * from workspace.gold.factbookings

In [0]:
df_gold = spark.sql(f"select * from {catalog}.{target_schema}.{target_object}")

df_gold.display()

In [0]:
df_gold_df = spark.read.format("csv")\
            .option("inferSchema",True)\
            .option('header', True)\
            .load("workspace.gold.dimairports")

In [0]:
df_gold_df.display()